# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [11]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'
print(myname)
print(MYTABLE)

damonchen
damonchen_data


In [12]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [9]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [13]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [15]:
url = "https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326"
my_data = pd.read_csv(url)
my_data.head()

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019


In [17]:
my_data.to_sql(myname + '_data', conn, index=True, if_exists="replace")

In [18]:
my_data.shape

(323, 21)

In [20]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dxyz.shape

(323, 22)

In [21]:
list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year']

True

In [22]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [ ]:
assert(type(answer) == list)
assert(answer == ['1194016923'])


## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [ ]:
total_payment=pd.read_sql_query("""
  SELECT m.total_payments, m.Business_ZIP_code, m.Medicaid_EP_Hospital_Type, p.Population, p.Zip
FROM 
    havenaspen_data m INNER JOIN
    population p ON m.business_zip_code = p.zip
GROUP BY Zip 
""", conn)
total_payment

In [ ]:
total_payment['personal_payment']=total_payment['total_payments']/total_payment['Population']
total_payment

In [ ]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [ ]:
answer=total_payment.loc[:, ['Medicaid_EP_Hospital_Type','personal_payment']]
answer

## Now using Pandas instead of SQL

In [ ]:
dhcs=pd.merge(tables['population'],tables['havenaspen_data'],how='right',left_on='Zip', right_on='Business_ZIP_Code')
dhcs
df=dhcs.loc[:, ['total_payments','Medicaid_EP_Hospital_Type','Population','Zip']]
df

In [ ]:
df['personal_payment']=df['total_payments']/df['Population']
answer=df
answer

In [ ]:
answer.drop_duplicates()



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [23]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


Already up to date.
[main f979717] Submitting the week 15 programming assignment
 2 files changed, 749 insertions(+), 27 deletions(-)
 create mode 100644 week15/week15_assignment.ipynb
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 4.44 KiB | 909.00 KiB/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   git@github.com:DamonChen-slu/hds5210-2022.git
To github.com:damonchen-slu/hds5210-2022.git
   582218c..f979717  main -> main
